## 1. Pengantar Database Relasional

Database relasional merupakan jenis *Database Management Systems* (DMBS) yang memberikan gambaran atau skema yang menjelaskan tentang hubungan antar tabel. 

Terdapat dua buah elemen penting yang digunakan untuk menghubungkan dua buah tabel, antara lain: __*primary key*__ dan __*foreign key*__. 

__*Primary key*__ merupakan kolom unik dalam tabel yang mengidentifikasikan sebuah *record*. Jika kolom *primary key* tabel "A" terdapat pada tabel "B", maka kolom *primary key* tabel "A" disebut sebagai "foreign key" pada tabel "B". __*Foreign key*__ digunakan untuk menghubungkan kedua tabel (*primary key* dihubungkan dengan *foreign key*-nya).


Pada praktek akan digunakan sampel database PostgreSQL yang dapat diperoleh pada tautan berikut: [Sample Database](https://sp.postgresqltutorial.com/wp-content/uploads/2019/05/dvdrental.zip). Database yang digunakan merupakan database rental DVD. Database rental DVD merepresentasikan proses bisnis toko rental DVD. Skema relasi database digambarkan pada gambar berikut:

![Database Rental DVD](https://sp.postgresqltutorial.com/wp-content/uploads/2018/03/dvd-rental-sample-database-diagram.png)

Penjelasan terkait tabel-tabel yang ditampilkan dalam skema tersebut adalah sebagai berikut:

* `actor` – menyimpan data identitas aktor.
* `film` – menyimpan data film.
* `film_actor` – menyimpan data relasi antara film dan aktor.
* `category` – menyimpan data kategori film.
* `film_category`- menyimpan data relasi antara film dan kategorinya.
* `store` – data toko termasuk manager toko dan alamat toko.
* `inventory` – menyimpan data inventarisasi.
* `rental` – menyimpan data peminjaman dvd.
* `payment` – menyimpan data pembayaran konsumen.
* `staff` – menyimpan data staf.
* `customer` – menyimpan data konsumen.
* `address` – menyimpan data alamat staf dan konsumen.
* `city` – menyimpan nama kota.
* `country` – menyimpan nama negara.

## 2. Workflow SQL query

Terdapat sejumlah langkah-langkah dasar untuk melakukan query database relasional, antara lain:

1. Import modul dan fungsi
2. Membuat database engine
3. Melakukan query dan menyimpan hasil query ke data frame

### 2.1. Import Modul

Terdapat tiga buah modul yang digunakan dalam praktek kali ini, antara lain:

1. `SQLAlchemy`,
2. `psycopg2`,
3. `pandas`

`SQLAlchemy` (SQA) menyediakan lapisan abstraksi di atas *raw SQL* yang dapat membantu kita untuk bekerja dengan tabel dan *query* sebagai objek di python. SQA dapat bekerja dengan banyak RDBMS, seperti: Postgresql, MySQL, dll. SQA biasanya dikombinasikan dengan modul lain yang berupa driver yang dapat membantu SQA membangun koneksi dengan sejumlah RDBMS. Untuk melakukan koneksi dengan database **Postgresql**, SQA memerlukan modul `psycopg2`.

Hasil *query* menggunakan bantuan kedua modul tersebut selanjutnya disimpan ke dalam data frame menggunakan modul `pandas`. `Pandas` merupakan modul populer yang sering digunakan untuk melakukan manipulasi data dan analisis data.

In [ ]:
# install SQLAlchemy
!pip install SQLAlchemy

In [ ]:
# install psycopg2
!pip install psycopg2

In [1]:
# import modul
from sqlalchemy import create_engine
import pandas as pd

### 2.2. Membuat Database Engine

Untuk mengakses database, langkah selanjutnya adalah membuat sql engine. Fungsi yang digunakan untuk mengakses sql engine adalah `create_engine()`. Skema fungsi tersebut adalah sebagai berikut:

```
# Skema: "<nama_db>+<driver>://<username>:<password>@<alamat_IP>:<port>/<nama_db>"
```


In [2]:
engine = create_engine('postgres+psycopg2://postgres:@localhost:5432/postgres')

Untuk mengecek tabel apa saja yang ada dalam database, jalankan metode `table_names()`.

In [3]:
tabel = engine.table_names()

print(tabel)

['actor', 'film', 'address', 'category', 'city', 'country', 'customer', 'film_actor', 'film_category', 'inventory', 'language', 'rental', 'payment', 'store', 'staff']


## 2.3. Query Database

Terdapat beberapa argumen utama yang digunakan untuk melakukan SQL *query*, antara lain:

* `SELECT` : memilih *field* atau kolom tabel
* `FROM` : memilih tabel
* `WHERE` : melakukan filter
* `GROUP BY` : melakukan aggregasi variabel
* `ORDER BY` : mengurutkan *record* berdasarkan suatu variabel
* `JOIN` : menggabungkan dua buah tabel dengan variasi, sebagai berikut:
    + `INNER JOIN`
    + `LEFT JOIN`
    + `RIGHT JOIN`
    + `FULL JOIN`

### 2.3.1. SELECT dan FROM

Format arggumen yang digunakan untuk memilih variabel dari suatu tabel adalah sebagai berikut:

```
SELECT kolom1, kolom2, ...
FROM nama_tabel
```

Jika kita ingin memilih seluruh kolom dalam suatu tabel, kita dapat mengganti nama kolom dengan karakter `*`

Berikut adalah contoh *query* untuk memperoleh data dari tabel `inventory`:


In [5]:
sql = """
SELECT *
FROM inventory
"""

df = pd.read_sql_query(sql, engine)

In [6]:
df

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 10:09:17
1,2,1,1,2006-02-15 10:09:17
2,3,1,1,2006-02-15 10:09:17
3,4,1,1,2006-02-15 10:09:17
4,5,1,2,2006-02-15 10:09:17
...,...,...,...,...
4576,4577,1000,1,2006-02-15 10:09:17
4577,4578,1000,2,2006-02-15 10:09:17
4578,4579,1000,2,2006-02-15 10:09:17
4579,4580,1000,2,2006-02-15 10:09:17


### 2.3.2. Melakukan Filter Pada Data

Format argumen yang dilakukan untuk melakukan filter pada data adalah sebagai berikut:

```
SELECT kolom1, kolom2, ...
FROM nama_tabel
WHERE kondisi
```

Misalkan kita ingin melakukan mengambil data pada tabel `customer` yang tidak memiliki *missing value* pada kolom `email`, berikut adalah sintaks yang digunakan:

In [7]:
sql = """
SELECT *
FROM customer
WHERE email IS NOT NULL
"""

df = pd.read_sql_query(sql, engine)

In [8]:
df

,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active
0,524,1,Jared,Ely,jared.ely@sakilacustomer.org,530,True,2006-02-14,2013-05-26 14:49:45.738,1
1,1,1,Mary,Smith,mary.smith@sakilacustomer.org,5,True,2006-02-14,2013-05-26 14:49:45.738,1
2,2,1,Patricia,Johnson,patricia.johnson@sakilacustomer.org,6,True,2006-02-14,2013-05-26 14:49:45.738,1
3,3,1,Linda,Williams,linda.williams@sakilacustomer.org,7,True,2006-02-14,2013-05-26 14:49:45.738,1
4,4,2,Barbara,Jones,barbara.jones@sakilacustomer.org,8,True,2006-02-14,2013-05-26 14:49:45.738,1
...,...,...,...,...,...,...,...,...,...,...
594,595,1,Terrence,Gunderson,terrence.gunderson@sakilacustomer.org,601,True,2006-02-14,2013-05-26 14:49:45.738,1
595,596,1,Enrique,Forsythe,enrique.forsythe@sakilacustomer.org,602,True,2006-02-14,2013-05-26 14:49:45.738,1
596,597,1,Freddie,Duggan,freddie.duggan@sakilacustomer.org,603,True,2006-02-14,2013-05-26 14:49:45.738,1
597,598,1,Wade,Delvalle,wade.delvalle@sakilacustomer.org,604,True,2006-02-14,2013-05-26 14:49:45.738,1


### 2.3.3. Aggregasi pada Tabel

Sering kali, kita hanya memerlukan data aggregasi berdasarkan variabel tertentu, format *query* yang digunakan adalah sebagai berikut:

```
SELECT kolom1, kolom2, ....
FROM nama_tabel
WHERE kondisi
GROUB BY kolom1, kolom2, ...
ORDER BY kolom1, kolom2,...
```

Misalkan kita membutuhkan data jumlah film yang rilis pada tiap tahunnya dari tabel `film`, sintaks yang digunakan adalah sebagai berikut:

In [14]:
sql = """
SELECT release_year, COUNT(release_year) AS jumlah_film
FROM film
GROUP BY release_year
ORDER BY jumlah_film DESC
"""

df = pd.read_sql_query(sql, engine)

In [15]:
df

,release_year,jumlah_film
0,2006,1000


### Menggabungkan Dua Buah Tabel

Suatu tabel sering-kali membutuhkan informasi lain untuk memudahkan kita membacanya, informasi lain dari suatu tabel biasanya tersimpan pada tabel lain yang memiliki korelasi dengannya. Informasi pada kedua tabel akan dihubungkan melalui kolom kunci yang ada pada kedua tabel. 

Join pada database dapat dilakukan dengan 4 cara:

* `INNER JOIN` : menggabungkan informasi berdasarkan intersep dari kedua tabel
* `LEFT JOIN` : menggabungkan seluruh baris pada tabel kiri dan sebagian baris pada tabel kanan yang elemen kuncinya cocok dengan tabel kiri
* `RIGHT JOIN` : kebalikan dari `LEFT JOIN`
* `FULL OUTER JOIN` : menggabungkan seluruh observasi pada kedua tabel melalui kolom elemen kunci.

![menggabungkan dua tabel](https://www.dofactory.com/Images/sql-joins.png)

Format argumen join adalah sebagai berikut:

```
SELECT tabel1.kolom1, tabel1.kolom2, tabel2.kolom1, ....
FROM tabel1
JOIN tabel2
ON tabel1.kolom1 = tabel2.kolom2
```

Gabungkan informasi yang ada pada tabel `category` dan `film_category`


In [20]:
sql = """
SELECT *
FROM category
FULL OUTER JOIN film_category ON category.category_id = film_category.category_id
"""

df = pd.read_sql_query(sql, engine)

In [21]:
df

,category_id,name,last_update,film_id,category_id,last_update
0,6,Documentary,2006-02-15 09:46:27,1,6,2006-02-15 10:07:09
1,11,Horror,2006-02-15 09:46:27,2,11,2006-02-15 10:07:09
2,6,Documentary,2006-02-15 09:46:27,3,6,2006-02-15 10:07:09
3,11,Horror,2006-02-15 09:46:27,4,11,2006-02-15 10:07:09
4,8,Family,2006-02-15 09:46:27,5,8,2006-02-15 10:07:09
...,...,...,...,...,...,...
995,6,Documentary,2006-02-15 09:46:27,996,6,2006-02-15 10:07:09
996,12,Music,2006-02-15 09:46:27,997,12,2006-02-15 10:07:09
997,11,Horror,2006-02-15 09:46:27,998,11,2006-02-15 10:07:09
998,3,Children,2006-02-15 09:46:27,999,3,2006-02-15 10:07:09


# Baca Lebih Lanjut

[SQL Tutorial](https://www.w3schools.com/sql/default.asp)